In [1]:
# reloads modules when running again
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
from data_exploration import *
from naive_pred import *
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from darts.models.forecasting.varima import VARIMA
from darts import TimeSeries
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor


import warnings
warnings.filterwarnings('ignore')

## Data loading

In [3]:
data, metadata = load_data()

## Data preprocessing

#### Predict by previous sample

In [4]:
# Add pseudo count
# numeric_cols = [col for col in data.columns if pd.api.types.is_numeric_dtype(data[col]) and col != "baboon_id"]
# ADDED_VALUE = 0.5 * data[numeric_cols][data[numeric_cols] > 0].min(axis = 1)
# data[numeric_cols] = data[numeric_cols].add(ADDED_VALUE, axis = 0)
# data[numeric_cols] = data[numeric_cols].div(data[numeric_cols].sum(axis = 1), axis = 0)

In [5]:
for max_gap in [7, 30, 60, 100]:
    X,y = shift_i_samples_remove_gaps(data.copy(), max_gap=max_gap)
    X = X.drop(columns = ["sample", "baboon_id", "collection_date"])
    y = y.drop(columns = ["sample", "baboon_id", "collection_date"])
    d_matrix = calc_distance_matrix(X,y, max_gap)
    plot_distances(d_matrix, max_gap)

#### ARIMA

In [6]:
numeric_cols = [col for col in data.columns if pd.api.types.is_numeric_dtype(data[col]) and col != "baboon_id"]
# is_stat = []
# for col in numeric_cols:
#     col_stat = []
#     col_pval = []
#     for baboon in data["baboon_id"].unique():
#         baboon_data = data[data["baboon_id"]==baboon]
#         col_stat.append(adfuller(baboon_data[col])[0])
#         col_pval.append(adfuller(baboon_data[col])[1])
#     is_stat.append(np.mean(col_pval) < 0.05)
#     print(f"\n{col}: mean ADF Statistic: {np.mean(col_stat)} mean pval: {np.mean(col_pval) < 0.05}")
# print(len([stat for stat in is_stat if stat]) / len(is_stat))

In [7]:
# y_indices = data.sort_values(["collection_date"]).groupby(["baboon_id"]).tail(5).index
# y = data[y_indices]
# X = data[~y_indices]

## training

#### Predict by previous sample

In [8]:
# """ VERY NAIVE MODEL """
# #-----------------------
# # no need to train, we use the previous sample as y_pred
# X = X.drop(columns = ["sample", "baboon_id", "collection_date"])
# y = y.drop(columns = ["sample", "baboon_id", "collection_date"])

#### ARIMA

In [9]:
pred_df = pd.DataFrame()

for baboon in data["baboon_id"].unique():
    baboon_data = data[data["baboon_id"]==baboon].drop(columns = ["sample", "baboon_id", "collection_date"])
    baboon_data["dummy_date"] = pd.date_range(datetime(year=2001, month=1,day=1), periods=len(baboon_data)).tolist()
    baboon_data = TimeSeries.from_dataframe(baboon_data, time_col="dummy_date")
    model = VARIMA() 
    model.fit(baboon_data)
    pred = model.predict(5)
    pred = pd.DataFrame(pred)
    pred["baboon_id"] = baboon
    pred_df = pd.concat([pred_df, pred], ignore_index=True)
    print(pred)
        
pred_df.to_csv("pred.csv")

LinAlgError: Matrix is not positive definite

## Performance analysis

In [ ]:
d_matrix = calc_distance_matrix(X,y)
plot_distances(d_matrix)